# Proyecto 1
## Detección de ataques

In [1]:
import pandas as pd
import numpy as np
import pandas_pro

In [2]:
part_1 = pd.read_csv("dataset/dataset-part1.csv")
part_2 = pd.read_csv("dataset/dataset-part2.csv")

df = pd.concat([part_1, part_2])

## Análisis Exploratorio

In [11]:
df

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8637202,1,0,"240,,,,,,,,,,,,,,240",0,1618262463,120,14015728,14015,1618262477935,1618262477,...,0,0,0,0,0,0,DNS.UbuntuONE,1,85888693,Normal flow
8637203,1,0,",",0,1618262477,120,0,0,1618262477940,1618262477,...,0,0,0,0,40,0,Unknown,1,85888694,Normal flow
8637204,1,0,"1259,592,52,,224",0,1618262473,120,4921754,4921,1618262477954,1618262477,...,1460,1410,7,7,40,0,SSH,1,85888695,Normal flow
8637205,1,0,205,0,1618262477,120,376,0,1618262477977,1618262477,...,0,0,0,0,128,0,DNS,1,85888696,Normal flow
